# Linear Regression Example with MINA-ZKML

In [ ]:
# Install required Python libraries
try:
    import subprocess
    subprocess.check_call(["pip", "install", "torch", "onnx", "numpy", "sklearn", "hummingbird-ml"])
except:
    print("Ensure all dependencies are installed.")

In [ ]:
from sklearn.linear_model import LinearRegression
from hummingbird.ml import convert
import numpy as np
import json
import torch
from sklearn.datasets import fetch_california_housing

# Load the housing price dataset
housing = fetch_california_housing()
X = housing.data
y = housing.target

# Train the model
reg = LinearRegression().fit(X, y)
print(f"Model coefficients: {reg.coef_}")
print(f"Model intercept: {reg.intercept_}")

In [ ]:
# Convert to PyTorch using Hummingbird
model = convert(reg, "torch", X[:1]).model

# Export to ONNX format
onnx_path = "models/linear_regression.onnx"
shape = X.shape[1:]
dummy_input = torch.rand(1, *shape, requires_grad=True)

torch.onnx.export(
    model, 
    dummy_input, 
    onnx_path,
    input_names=["input"], 
    output_names=["output"],
    opset_version=13
)
print(f"ONNX model exported to {onnx_path}")

## Installation and Configuration

Now that we have our model trained and exported to ONNX format, let's set up the zkML verification system. First, we'll install and configure the Mina zkApp CLI:

In [ ]:
# Install Mina zkApp CLI
!npm install -g zkapp-cli

# Install project dependencies
!npm install

print("Dependencies installed successfully")

### Configure Mina zkApp

Now we'll set up the zkApp configuration. Run each step and follow the prompts:

1. Initialize zkApp configuration with interactive prompts:
   - Name: Choose 'devnet'
   - Network: Choose 'Testnet'
   - API URL: Use 'https://api.minascan.io/node/devnet/v1/graphql'
   - Fee: Set to 0.1 MINA
   - Account: Choose or create a fee payer account

After configuration, you'll need to request testnet MINA from the faucet before deployment.

In [ ]:
# Initialize zkApp configuration
!zk config

print("\nAfter configuration:")
print("1. Note your fee payer account public key from the output above")
print("2. Visit https://faucet.minaprotocol.com/?address=<YOUR-FEE-PAYER-ACCOUNT>")
print("3. Select 'Devnet' network and request funds")
print("4. Wait a few minutes for the transaction to be included in a block")

### Model Configuration

While waiting for the testnet funds, let's create our model configuration:

In [ ]:
# Create model configuration
model_config = {
    "modelConfig": {
        "modelPath": "models/linear_regression.onnx",
        "variables": {
            "batch_size": 1
        },
        "visibility": {
            "input": "Public",
            "output": "Private"
        },
        "defaultInput": [sample_input.flatten().tolist()]  # Using our sample input from the dataset
    }
}

with open("modelConfig.json", "w") as f:
    json.dump(model_config, f, indent=2)

# Save sample input data for verification
sample_data = {
    "input": X[0].tolist()  # Using first sample from training data
}

with open("sample_data.json", "w") as f:
    json.dump(sample_data, f, indent=2)

print("Model and sample data configurations created successfully")

## Generating and Verifying Proofs

Let's generate a proof using our trained model:

In [ ]:
# Generate proof using generateProof.js
# This will use the defaultInput from modelConfig.json
!node generateProof.js models/linear_regression.onnx

print("Proof generation completed")

# Save proof data to output.json
!mv proof_data.json output.json

## Using the API Service

Now let's verify our proof using the API service:

In [ ]:
# Create TypeScript verification script
verify_script = """
import axios, { AxiosError } from 'axios';
import fs from 'fs';

interface ProofResponse {
  status: string;
  transactionUrl?: string;
  error?: string;
}

interface SampleData {
  input: number[];
}

const proofPath = 'output.json';
const sampleDataPath = 'sample_data.json';

async function testProofVerification() {
  try {
    // Read the proof data and sample input
    const proofData = await fs.readFileSync(proofPath, 'utf8');
    const sampleData: SampleData = JSON.parse(await fs.readFileSync(sampleDataPath, 'utf8'));

    // Send verification request with actual training data
    const response = await axios.post<ProofResponse>(
      'http://localhost:3000/api/verify-proof', 
      {
        proofData: proofData,
        publicInput: [sampleData.input]  // Using actual training data
      }
    );

    console.log('Proof verification result:', response.data);
  } catch (error) {
    if (axios.isAxiosError(error)) {
      const axiosError = error as AxiosError<ProofResponse>;
      console.error('Proof verification failed:', axiosError.response?.data);
    } else if (error instanceof Error) {
      console.error('Error:', error.message);
    } else {
      console.error('Unknown error occurred');
    }
  }
}

// Run the test
testProofVerification();
"""

with open('verify.ts', 'w') as f:
    f.write(verify_script)

# Run verification script
!node verify.ts

## Smart Contract Integration

Now let's create and deploy our verifier contract. Make sure you've received your testnet funds before proceeding with deployment.

In [ ]:
# Create smart contract
contract_code = """
import { ZKMLVerifier, ZKMLProofData } from 'zkml-verifier';

export class CustomVerifier extends ZKMLVerifier {
    @method async verifyModelPrediction(proofData: ZKMLProofData) {
        // Set up verifier
        this.setVerifierSystem(proofData.verifierSystem);
        this.setProofData(proofData.proof);
        
        // Verify the proof
        const isValid = await this.verifyProof();
        isValid.assertTrue();
    }
}
"""

with open('CustomVerifier.ts', 'w') as f:
    f.write(contract_code)

# Compile contract
!npx tsc CustomVerifier.ts

print("Smart contract created and compiled successfully")

In [ ]:
# Deploy contract to Mina devnet
# Make sure you've received your testnet funds before running this
!zk deploy devnet

print("\nContract deployment status:")
print("1. Check the transaction hash in the output above")
print("2. View your deployment on https://minascan.io/devnet/tx/<TRANSACTION-HASH>")
print("3. Wait for the transaction to be included in a block (usually a few minutes)")